In [4]:
import pickle
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import shuffle

from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms

train_path = "/Users/dpap/Documents/UvA/AML/Assignments/Kaggle/pickle/initial_dataset/"
test_path = "/Users/dpap/Documents/UvA/AML/Assignments/Kaggle/pickle/initial_dataset/"

In [5]:
!ls pickle/initial_dataset/

test_images.pkl  train_images.pkl val_fname.pkl
test_labels.pkl  train_labels.pkl val_images.pkl


# Data

In [22]:
train_images = pickle.load(open(train_path+"train_images.pkl","rb"))
train_labels = pickle.load(open(train_path+"train_labels.pkl","rb"))

test_images  = pickle.load(open(train_path+"test_images.pkl","rb"))
test_labels  = pickle.load(open(train_path+"test_labels.pkl","rb"))

val_images   = pickle.load(open(train_path+"val_images.pkl","rb"))
val_fname    = pickle.load(open(train_path+"val_fname.pkl","rb"))

# Datasets and Dataloaders

In [7]:
class TrainingImagesDataset(Dataset):
    
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        single_image = self.images[index]
        single_label = self.labels[index]
        
        if self.transform != None:
            #image to tensor
            tensor = self.transform(single_image)
            return (tensor, single_label)
        #return image without turning it to tensor
        #probably wrong
        return (single_image, single_label)

In [8]:
class ValidationImagesDataset(Dataset):
    
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        single_image = self.images[index]
        if self.transform:
            #image to tensor
            tensor = self.transform(single_image)
            return tensor
        return single_image

In [9]:
def create_dataloaders(train_images, train_lbs, test_images=None, test_lbs = None, batch_size = 32):
    
    train_dataset = TrainingImagesDataset(train_images, train_lbs, transform=transforms.Compose([transforms.ToTensor()]))
    
    if test_images and test_lbs:
        print("here")
        test_dataset = TrainingImagesDataset(test_images, test_lbs, transform=transforms.Compose([transforms.ToTensor()]))
    elif test_images:
        test_dataset = ValidationImagesDataset(test_images, transform=transforms.Compose([transforms.ToTensor()]))
    else:
        test_dataset = []
    
    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 0)
    test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False, num_workers = 0)    
    
    return (train_loader, test_loader)

# Neural Network

In [20]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 12, 5, padding=2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 32, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 48, 3, padding=1)
        #self.pool3 = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(48,32,3, padding=1)
        self.pool3 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(2048, 2048)
        self.fc2 = nn.Linear(2048, 2048)
        self.fc3 = nn.Linear(2048, 121)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool3(F.relu(self.conv4(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


ocean = CNN()

# Training

In [27]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(ocean.parameters(), lr = 0.001, momentum = 0.9)#, weight_decay=0.0005)

In [28]:
train_loader, test_loader = create_dataloaders(train_images, train_labels, test_images, test_labels)

here


In [ ]:
%%time
EPOCH = 25
for epoch in range(EPOCH):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, lbs = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = ocean(inputs)
        loss = criterion(outputs, lbs)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 32 == 31:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 32))
            running_loss = 0.0

print('Finished Training')

[1,    32] loss: 4.794
[1,    64] loss: 4.786
[1,    96] loss: 4.779
[1,   128] loss: 4.768
[1,   160] loss: 4.764
[1,   192] loss: 4.753
[1,   224] loss: 4.745
[1,   256] loss: 4.721
[1,   288] loss: 4.682
[1,   320] loss: 4.550
[1,   352] loss: 4.259
[1,   384] loss: 4.195
[1,   416] loss: 4.155
[1,   448] loss: 4.225
[1,   480] loss: 4.192
[1,   512] loss: 4.195
[1,   544] loss: 4.185
[1,   576] loss: 4.147
[2,    32] loss: 4.138
[2,    64] loss: 4.143
[2,    96] loss: 4.150
[2,   128] loss: 4.154
[2,   160] loss: 4.172
[2,   192] loss: 4.171
[2,   224] loss: 4.134
[2,   256] loss: 4.144
[2,   288] loss: 4.069
[2,   320] loss: 4.115
[2,   352] loss: 4.066
[2,   384] loss: 3.939
[2,   416] loss: 4.027
[2,   448] loss: 4.009
[2,   480] loss: 3.995
[2,   512] loss: 3.946
[2,   544] loss: 3.933
[2,   576] loss: 3.945
[3,    32] loss: 3.931
[3,    64] loss: 3.911
[3,    96] loss: 3.955
[3,   128] loss: 3.890
[3,   160] loss: 3.828
[3,   192] loss: 3.852
[3,   224] loss: 3.843
[3,   256] 